In [1]:
import numpy as np
import pandas as pd

In [2]:
def get_data():
    df = pd.read_csv('ecommerce_data.csv')
    data = df.as_matrix()
    print df.head()
    X = data[:, : -1]
    Y = data[:, -1] 
    
    X[:, 1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()
    X[:, 2] = (X[:,2] - X[:,2].mean()) / X[:,2].std()
    
    N, D = X.shape
    X2 = np.zeros((N, D+3))
    X2[:, : D-1] = X[:, : D-1]
    
    for n in xrange(N):
        t = X[n, D-1]
        X2[n, t+ D-1] = 1
    
    #print X2
    
    Z = np.zeros((N, 4))
    Z[np.arange(N), X[:, D-1].astype(np.int32)] = 1
    #print Z
    
    assert(np.abs(X2[:, -4:] - Z).sum() < 10e-10)
    return X2, Y

def get_binary_data():
    X, Y = get_data()
    X2 = X[Y <= 1]
    Y2 = Y[Y <= 1]
    return X2, Y2

In [3]:
X, Y = get_data()
D = X.shape[1]
M = 5
K = len(set(Y))

W1 = np.random.randn(D, M)
b1 = np.zeros(M)
W2 = np.random.randn(M, K)
b2 = np.zeros(K)

   is_mobile  n_products_viewed  visit_duration  is_returning_visitor  \
0          1                  0        0.657510                     0   
1          1                  1        0.568571                     0   
2          1                  0        0.042246                     1   
3          1                  1        1.659793                     1   
4          0                  1        2.014745                     1   

   time_of_day  user_action  
0            3            0  
1            2            1  
2            1            0  
3            1            2  
4            1            2  


/Library/Python/2.7/site-packages/ipykernel/__main__.py:17: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [4]:
def softmax(a):
    expA = np.exp(a)
    return expA / expA.sum(axis = 1, keepdims=True)

def forward(X, W1, b1, W2, b2):
    Z = np.tanh(X.dot(W1) + b1)
    A = Z.dot(W2) + b2
    Y = softmax(A)
    return Y

def classification_rate(Y, P):
    n_correct = 0
    n_total = 0
    for i in xrange(len(Y)):
        n_total +=1
        if (P[i] == Y[i]):
            n_correct +=1
        
    return float(n_correct) / n_total


In [5]:
P_Y_given_X = forward(X, W1, b1, W2, b2)
P = np.argmax(P_Y_given_X, axis = 1)
rate = classification_rate(Y, P)
print P.shape
print "Score :", rate

(500,)
Score : 0.224
